In [2]:
# ✅ 1. Import Libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ✅ 2. Load & Clean Data
df = pd.read_csv('/home/deva_kush/ml_projects/CKD/data/kidney_disease.csv')

# Drop 'id' column if it exists
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("'", "")

# Replace missing values
df.replace(['?', '', 'NA', 'na', 'NaN', 'nan', ' '], np.nan, inplace=True)

# Rename class column if needed
if 'class' in df.columns:
    df.rename(columns={'class': 'classification'}, inplace=True)

# Lowercase and strip target labels
df['classification'] = df['classification'].astype(str).str.strip().str.lower()

# Drop columns and rows with more than 50% missing values
df.dropna(axis=1, thresh=int(0.5 * len(df)), inplace=True)
df.dropna(axis=0, thresh=int(0.5 * len(df.columns)), inplace=True)

# Fill remaining NaNs with median or mode
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].median(), inplace=True)

# ✅ 3. Encode Categorical Columns
df_encoded = df.copy()
encoders = {}

for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        encoders[col] = le

# ✅ 4. Train-Test Split
X = df_encoded.drop('classification', axis=1)
y = df_encoded['classification']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ 5. Train Model
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# ✅ 6. Evaluate Model
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("✅ Classification Report:\n", classification_report(y_test, y_pred))

# ✅ 7. GUI Fields
input_fields = X.columns.tolist()
widgets_list = []
user_widgets = {}

# Create input widgets
for field in input_fields:
    if field in encoders and field != 'classification':
        opts = list(encoders[field].classes_)
        user_widgets[field] = widgets.Dropdown(
            options=opts, description=field.upper()+":", style={'description_width': 'initial'}
        )
    else:
        user_widgets[field] = widgets.FloatText(
            value=float(df[field].median()), description=field.upper()+":", style={'description_width': 'initial'}
        )
    widgets_list.append(user_widgets[field])

# ✅ 8. Predict Button & Output
btn = widgets.Button(description="🔍 Predict CKD", button_style='success')
output = widgets.Output()
widgets_list.append(btn)
widgets_list.append(output)

# ✅ 9. Prediction Logic
def on_click(b):
    with output:
        clear_output()

        input_data = {}
        for field in input_fields:
            val = user_widgets[field].value
            if field in encoders:
                try:
                    val = encoders[field].transform([val])[0]
                except:
                    val = encoders[field].transform([encoders[field].classes_[0]])[0]
            input_data[field] = val

        df_input = pd.DataFrame([input_data])
        prediction = model.predict(df_input)[0]
        class_label = encoders['classification'].inverse_transform([prediction])[0]
        confidence = model.predict_proba(df_input)[0][prediction] * 100

        print("🔮 Prediction Result:")
        print(f"✅ Class: {class_label.upper()}")
        print(f"📊 Confidence: {confidence:.2f}%")

btn.on_click(on_click)

# ✅ 10. Show GUI
display(widgets.VBox(widgets_list))

/tmp/ipykernel_1321/660511826.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_1321/660511826.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

✅ Accuracy: 1.0
✅ Confusion Matrix:
 [[52  0]
 [ 0 28]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        52
           1       1.00      1.00      1.00        28

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

